<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing II</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=463">MICRO-512</a>) taught by Dr. D. Sage, Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2024.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:sebastien.herbreteau@epfl.ch">Sébastien Herbreteau</a> and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
     
</p>
<hr style="clear:both">
<h1>Lab 7.1: Deconvolution  </h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday May 23, 2024</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Monday June 3, 2024</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=463">Moodle</a></p>
    <p style="margin:4px;"><b>Total number of points</b>: 21</p>
    <p style="margin:4px;"><b>Helps session</b>: 10:15 - 12:00, Thursday May 30, 2024 in CM 2</p>     
    <p style="margin:4px;"><b>Related lectures</b>: Chapters 9 and 10</p>
</div>

Double-click on this cell, fill your name and SCIPER number below to verify your identity in Noto and set the seed for random results.
:::{attention} Please write down your name and SCIPER! 
### Student Name: 
### SCIPER: 
:::

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## <a name="imports_"></a> Imports
In the next cell we import Python libraries we will use throughout the lab.

In [ ]:
# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import skimage
from interactive_kit import imviewer as viewer

# Configure plotting as dynamic
%matplotlib widget

# Deconvolution (14 points)

In this lab, you will study deconvolution algorithms. You will implement by yourself simple deconvolution approaches, and explore more regularization techniques with Pyxu in the second lab.

## <a id="ToC_1_NN"></a>Table of contents
1. [Inverse filtering](#1.-Inverse-filtering)
    1. [Reproducing blur on a test image](#1.A.-Reproducing-blur-on-a-test-image) **(2 points)**
    2. [Revealing the identity of the blurred celebrity](#1.B.-Revealing-the-identity-of-the-blurred-celebrity)  **(2 points)**
    3. [Robustness to mirror padding](#1.C.-Robustness-to-mirror-padding) **(1 point)**
2. [Regularization](#2.-Regularization)
    1. [Tikhonov regularization](#2.A.-Tikhonov-regularization) **(5 points)**
    2. [Wavelet regularization](#2.B.-Wavelet-regularization) **(4 points)**
    3. [TV regularization](#2.C.-TV-regularization) **(optional - 0 point)**

## 1. Inverse filtering
[Back to table of contents](#ToC_1_NN)

In order to preserve the identity of a (famous) person in an image $s$, a naive software engineer at a Big Tech company decides to blur the entire image $s$ with a centered Gaussian kernel, denoted $h$. In other words, the corruption model he uses is simply: $$y = h * s\,,$$
where $s$ is the true signal, $h$ is the blurring kernel (with **periodic padding**) and $y$ the corrupted image. The blurred image $y$ is available at `images/blurred.tif`.

Before revealing the identity of the person in `blurred.tif` in part [1.B](#1.B.-Revealing-the-identity-of-the-blurred-celebrity), you are going to experiment first with the test image `einstein.tif`.

### 1.A. Reproducing blur on a test image
[Back to table of contents](#ToC_1_NN)

For **1 point**, complete the function `filtering_periodic` that implements the operation $h * s$ with periodic padding **using 2D discrete Fourier transform (DFT)** and test it with the test image of Albert Einstein (`images/einstein.tif`) and the given centered Gaussian kernel $h$ with standard deviation $\sigma_h=20$. You should use the NumPy functions `np.fft.fft2` and `np.fft.ifft2`.

The function `filtering_periodic` takes as inputs
 * `s`: the input image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 
and returns `y`, the filtered image of the same size as `s`.

<div class="alert alert-info">
<b>Hint:</b> you may want to use the given function <code>kernel_to_image</code> below.
</div>
    
<div class="alert alert-danger">
<b>Beware:</b> <code>filtering_periodic</code> must output an image with pixels in $\mathbb{R}$ (not in $\mathbb{C}$).
</div>

In [ ]:
def get_2D_Gaussian_kernel(sigma_h):
    """ Returns a centered Gaussian kernel h of size (2k+1)*(2k+1) """
    k = int(np.ceil(3*sigma_h))
    kernel_h_1D = np.exp(-np.arange(-k, k+1)**2 / (2 * sigma_h**2))
    kernel_h_1D /= kernel_h_1D.sum() # normalization so that the sum of the coefficients equals 1
    kernel_h_2D = kernel_h_1D.reshape(-1, 1) @ kernel_h_1D.reshape(1, -1)
    return kernel_h_2D
    
def kernel_to_image(h, s):
    """ Converts the kernel h of size (2k+1)x(2k+1) to an image of the same size as s, denoted h_ext, 
    so that the relation h * s = y <=> DFT(h_ext) .* DFT(s) = DFT(y) holds """
    h_ext = np.zeros(s.shape)
    k = h.shape[0] // 2
    h_ext[:(2*k+1), :(2*k+1)] = h
    return np.roll(h_ext, (-k, -k), axis=(0, 1))
    
def filtering_periodic(s, h):
    """ Returns h * s """
    y = None
    # YOUR CODE HERE
    return y

In [ ]:
# Import Einstein image
einstein = skimage.io.imread('images/einstein.tif').astype(np.float64)

# Display result
sigma_h = 20
kernel_h_2D = get_2D_Gaussian_kernel(sigma_h)
blurred_einstein = filtering_periodic(einstein, kernel_h_2D)
plt.close('all')
view = viewer([einstein, blurred_einstein, kernel_h_2D], widgets=True, hist=False, axis=True, cmap='gray')

Although it may seem that the person has been anonymized, this way of corrupting data is not safe as it is completely reversible as long as the Gaussian kernel $h$ is known. For **1 point**, implement the function `inverse_filtering_periodic` **using 2D discrete Fourier transform (DFT)** that recovers the true signal $s$ from its corrupted version $y$.


The function `inverse_filtering_periodic` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 
and returns `s`, the original image of the same size as `y`.

In [ ]:
def inverse_filtering_periodic(y, h):
    """ Returns s such that y = h * s """
    s = None
    # YOUR CODE HERE
    return s

deblurred_einstein = inverse_filtering_periodic(blurred_einstein, kernel_h_2D)
absolute_error = np.abs(deblurred_einstein - einstein) # due to numerical errors
plt.close('all')
view = viewer([deblurred_einstein, absolute_error], widgets=True, hist=False, axis=True, cmap='gray')

In [ ]:
# Perform sanity check on a random image
s_random = np.random.rand(40, 40)
h_random = np.random.rand(5, 5)
if np.allclose(inverse_filtering_periodic(filtering_periodic(s_random, h_random), h_random), s_random) \
    and np.max(np.abs(np.imag(filtering_periodic(s_random, h_random)))) == 0 \
    and np.max(np.abs(np.imag(inverse_filtering_periodic(s_random, h_random)))) == 0:
    print('Congratulations! Your code seems to be working.') 
else:
    print('WARNING!\nYour code is not correct!') 

### 1.B. Revealing the identity of the blurred celebrity
[Back to table of contents](#ToC_1_NN)

For **2 points**, using the function `inverse_filtering_periodic`, reveal the identity of the blurred person at `images/blurred.tif`, given that the standard deviation $\sigma_h$ of the Gaussian kernel used for corruption is **an integer**. 

<div class="alert alert-info">
<b>Hint:</b> supply a list of potential $\sigma_h$ (stored in <code>list_sigmas</code>), run the deconvolution for each $\sigma_h$ and store the images in <code>list_deblurred_images</code>.
</div>

In [ ]:
blurred_person = skimage.io.imread('images/blurred.tif').astype(np.float64)

list_sigmas = [0]
list_deblurred_images = [blurred_person]

# YOUR CODE HERE

plt.close('all')
view = viewer(list_deblurred_images, title=[str(elt) for elt in list_sigmas], widgets=True, hist=False, axis=True, cmap='gray')

In the next cell, assign to the variable `sigma_blur` the standard deviation used for blurring the hidden person.

In [ ]:
# Standard deviation used for blurring the hidden celebrity
sigma_blur = None
# YOUR CODE HERE

In [ ]:
# Perform sanity check on sigma_blur
if not (0 < sigma_blur < 50 or np.round(sigma_blur) != sigma_blur): 
    print('WARNING!\nThe selected standard deviation is not really reasonable.')

### 1.C. Robustness to mirror padding
[Back to table of contents](#ToC_1_NN)

For **1 point**, **using the function** `filtering_periodic`, implement the function `filtering_mirror` that filters the input image with a mirror padding (a.k.a. reflect padding) and observe that the function `inverse_filtering_periodic` defined above does not work for deblurring with this specific padding. 

The function `filtering_mirror` takes as inputs
 * `s`: the input image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 
and returns `y`, the filtered image with mirror padding of the same size as `s`.

$\Rightarrow$ Inverse filtering is not robust to a slight change in the padding mode!


<div class="alert alert-info">
<b>Hint:</b> You may want to use the function <code>np.pad</code> with reflect mode.
</div>

In [ ]:
def filtering_mirror(s, h):
    """ Returns h * s with mirror padding """ 
    y = None
    # YOUR CODE HERE
    return y

In [ ]:
blurred_einstein_mirror = filtering_mirror(einstein, kernel_h_2D)
deblurred_einstein_mirror = inverse_filtering_periodic(blurred_einstein_mirror, kernel_h_2D)
plt.close('all')
view = viewer([deblurred_einstein_mirror, blurred_einstein_mirror], widgets=True, hist=False, axis=True, cmap='gray')

## 2. Regularization
[Back to table of contents](#ToC_1_NN)

In order to make it more difficult for potential attackers, the engineer decides to add white Gaussian noise $n \sim \mathcal{N}(0, \sigma_n^2 I)$ to the corruption model (still periodic padding):
$$y = h * s + n\,,$$
with $\sigma_n = 1$ (almost imperceptible noise). 

Observe that the function `inverse_filtering_periodic` defined above is not enough to recover the true signal $s$.

$\Rightarrow$ Inverse filtering is not robust to the addition of a weak noise!

In [ ]:
np.random.seed(1234) # for reproductibility, do not modify
blurred_einstein = filtering_periodic(einstein, kernel_h_2D) 
blurred_and_weakly_noisy_einstein = blurred_einstein + 1.0 * np.random.randn(*einstein.shape)
recovered_einstein = inverse_filtering_periodic(blurred_and_weakly_noisy_einstein, kernel_h_2D)
plt.close('all')
view = viewer([blurred_einstein, blurred_and_weakly_noisy_einstein, recovered_einstein], widgets=True, hist=False, axis=True, cmap='gray')

To tackle this problem, the least-squares method consists in solving:
$$s^\ast = \arg \min_s J_{\text{LS}}(s, y)\,,$$
where $J_{\text{LS}}(s, y) = \|h * s - y\|_2^2$. Note that, in this case, the least-squares solution coincides with the maximum-likelihood (ML) estimation solution. However, from a global perspective, ML deconvolution is ill-posed. It is better to constrain the solution by imposing explicit regularization constraints:
$$s^\ast_\lambda = \arg \min_s J_{\text{LS}}(s,y) + \lambda J_\text{reg}(s)\,,$$
where $\lambda \geq 0$ is an hyperparameter.

### 2.A. Tikhonov regularization
[Back to table of contents](#ToC_1_NN)

In one of its simplest form, Tikhonov regularization is obtained with $J_\text{reg}(s) = \|Ls\|_2^2$ where $L$ is the Laplacian operator that penalizes oscillations.  For **2 points**, complete the function `solve_tikhonov` that computes the **formal solution** $s^\ast_\lambda$ using **discrete Fourier transform (DFT)**.


The function `solve_tikhonov` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 * `lam` (scalar): the hyperparameter $\lambda \geq 0$, 
 
and returns `solution`, the formal solution $s^\ast_\lambda$.


<div class="alert alert-info">
<b>Hints:</b>
    
1. The gradient of $J_{\text{LS}}(s,y) + \lambda J_\text{reg}(s)$ with regard to $s$ is equal to $2( (H^\top H + \lambda L^\top L) s -  H^\top y)$ where $H$ is the matrix formulation of convolution with kernel $h$ (in the discretized forward model). **Set it to zero to get the linear system for which $s^\ast_\lambda$ is solution**, then go to Fourier.
    
2. For the sake of simplicity, we use the $3 \times 3$ Laplacian operator, defined as $Ls = l * s$ with $$l = \begin{pmatrix} 0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0 \\
\end{pmatrix}\,.$$
    
3. Since all convolutions have symmetric kernels, $H^\top = H$ and $L^\top = L$.
</div>

In [ ]:
def solve_tikhonov(y, h, lam):
    """ Returns the formal solution s_\lambda^\ast """
    solution = None
    # YOUR CODE HERE
    return solution

In [ ]:
# Perform sanity check for solve_tikhonov
np.random.seed(5678) # for reproductibility, do not modify
y_sanity_check = np.random.rand(16, 16)
h_sanity_check = np.array([[0.05, 0.1, 0.05], [0.1, 0.4, 0.1], [0.05, 0.1, 0.05]])
lam_sanity_check = 1e-1
solution_sanity_check = np.array([[0.43401456, 0.37066355, 0.47667136, 0.5514082 ],
       [0.56186321, 0.42420904, 0.5007256 , 0.49477319],
       [0.62218151, 0.43939143, 0.53553075, 0.53750101],
       [0.58575311, 0.47531916, 0.54754149, 0.59132934]])
if np.allclose(solution_sanity_check, solve_tikhonov(y_sanity_check, h_sanity_check, lam_sanity_check)[:4, :4]):
    print('Congratulations! Your function "solve_tikhonov" seems to be working.') 
else:
    print('WARNING!\nYour function "solve_tikhonov" is not correct!')

Observe the effet of the regularization hyperparameter $\lambda$ for deblurring Einstein image $y$.

<div class = 'alert alert-success'> <b>Reminder:</b> <code>Extra Widgets</code> needs to be clicked to see the effect of deconvolution with <code>IntSlider</code>.
</div>

In [ ]:
# Declare slider
reg_slider = widgets.IntSlider(value=-8, min=-8, max=8, step=1, description='lam = 10^x')
activation_button = widgets.Button(description='Regularize')

# Declare callback
def callback(img):
    reg = reg_slider.value
    output = solve_tikhonov(img, kernel_h_2D, 10**reg)
    return output

plt.close('all')
view = viewer(blurred_and_weakly_noisy_einstein, new_widgets=[reg_slider, activation_button], callbacks=[callback], widgets=True)

The **signal to noise ratio** (SNR), expressed in decibels (dB), is defined between a ground truth image $s$ and its recovered version $\hat{s}$ as follows:
    $$\operatorname{SNR}(s,\hat{s}) = 10 \log_{10} \left(\frac{\sum_{i} s_i^2}{\sum_{i} (s_i - \hat{s}_i)^2}\right)\,.$$
Complete the function `SNR` below that implements it. The function `SNR` takes as inputs

 * `s`: the ground truth image, 
 * `s_hat`: the estimated image, 
 
and returns `snr` (float), the signal to noise ratio.

In [ ]:
def SNR(s, s_hat):
    snr = None
    # YOUR CODE HERE
    return snr

In [ ]:
# Sanity check for the function SNR
s1 = np.linspace(0.8, 1, 10).reshape(-1, 1) @ np.linspace(0.9, 1, 10).reshape(1, -1)
s2 = np.linspace(0.85, 1, 10).reshape(-1, 1) @ np.linspace(0.99, 1, 10).reshape(1, -1)
if np.allclose(SNR(s1, s2), 21.58120) and np.allclose(SNR(s2, s1), 22.20615):
    print('Congratulations! Your function "SNR" seems to be working.') 
else:
    print('WARNING!\nYour function "SNR" is not correct!')

For **1 point** answer the following MCQ:

* Among the following values of $\lambda$, which one maximizes the SNR between $s$ and $s^\ast_\lambda$ ?
1. $\lambda = 10^{-6}$
2. $\lambda = 10^{-4}$
3. $\lambda = 10^{-2}$
4. $\lambda = 10^{0}$
5. $\lambda = 10^{2}$
6. $\lambda = 10^{4}$
7. $\lambda = 10^{6}$

Use the next cell to experiment or confirm your hypothesis and then modify the variable `answer` in the cell right after it to reflect your choices. 

In [ ]:
# YOUR CODE HERE

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

#### Behavior at the limits

**For 1 point**, answer the following MCQ:
* What is the SNR of the least-squares solution, i.e. when $\lambda=0$ (rounded to the nearest integer) ? (**1 point**)
1. $-514$ dB
2. $-414$ dB
3. $-314$ dB
4. $-214$ dB
5. $-114$ dB
6. $0$ dB
7. $14$ dB

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

**For 1 point** answer the following MCQ:
* What is $s_\lambda^\ast$ when $\lambda$ tends to infinity ? 

**Note:** $\mathbf{0}$ and $\mathbf{1}$ denote the all-zeros vector and the all-ones vector of the same size as $y$, respectively.

1. $y$
2. $\mathbf{0}$
3. $\mathbf{1}$
4. $Hy$
5. $H \mathbf{1}$
6. $\operatorname{mean}(y) \mathbf{1}$
7. $\operatorname{median}(y) \mathbf{1}$

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

### 2.B. Wavelet regularization
[Back to table of contents](#ToC_1_NN)

Wavelet regularization is obtained by setting $J_\text{reg}(s) = \|W s\|_1$ where $W$ is a suitable wavelet transform (favors "sparse" solutions). For the sake of simplicity, we propose to use Haar wavelets, for which the transform and the inverse transform are given below (scale=3 by default, do not modify it).

In [ ]:
def transform_wavelet_haar(img, scale=3):
    if scale==0:
        return img
    h, w = img.shape
    img_transform = np.zeros_like(img)
    a, b, c, d = img[::2, ::2], img[1::2, ::2], img[::2, 1::2], img[1::2, 1::2]
    img_transform[:h//2, :w//2] = transform_wavelet_haar((a + b + c + d) / 2, scale=scale-1)
    img_transform[:h//2, w//2:] = (a + b - c - d) / 2
    img_transform[h//2:, :w//2] = (a - b + c - d) / 2
    img_transform[h//2:, w//2:] = (a - b - c + d) / 2
    return img_transform

In [ ]:
def inverse_transform_wavelet_haar(img, scale=3):
    if scale==0:
        return img
    h, w = img.shape
    img_transform = np.zeros_like(img)
    a, b, c, d = inverse_transform_wavelet_haar(img[:h//2, :w//2], scale=scale-1), img[:h//2, w//2:], img[h//2:, :w//2], img[h//2:, w//2:]
    img_transform[::2, ::2] =  (a + b + c + d) / 2
    img_transform[1::2, ::2] = (a + b - c - d) / 2
    img_transform[::2, 1::2] = (a - b + c - d) / 2
    img_transform[1::2, 1::2] = (a - b - c + d) / 2
    return img_transform

In [ ]:
# Display of those functions
w_transform = transform_wavelet_haar(einstein)
inv_w_transform = inverse_transform_wavelet_haar(w_transform)
plt.close('all')
view = viewer([w_transform, inv_w_transform], widgets=True, hist=False, axis=True, cmap='gray')

**Optimization with Iterative Soft-Thresholding Algorithm (ISTA)**: 

**ISTA** (Figueiredo-Nowak, 2003) is a special case of the **Proximal Gradient Descent** algorithm (a.k.a. Forward-backward splitting algorithm, Combettes-Wajs, 2005) which is summarized here, for the sake of completeness. General optimization problem:

$$s^\ast = \arg \min_s f(s) + g(s)$$
with $f$ convex and differentiable with $L$-Lipschitz continuous gradient and $g$ convex, not necessarily differentiable.

The solution by **Proximal Gradient Descent** is given by repeating:
$$
\left\{
    \begin{array}{ll}
        z^{(n)} &= s^{(n-1)} - \gamma \nabla f (s^{(n-1)}) \quad \text{(gradient step)} \\
        s^{(n)} &= \operatorname{prox}_{\gamma g}(z^{(n)}) \quad \quad \quad \quad \text{(proximal step)}
    \end{array}
\right.
$$
with fixed step size $\gamma \leq 1/L$ and where the proximal operator is defined as:
$$\operatorname{prox}_{g}(z) = \arg \min_s \frac{1}{2} \| z - s\|_2^2 + g(s)\,.$$

<div class = 'alert alert-success'>
<b> Reminder: </b>In the case of <b>wavelet regularization</b>, we have for <b>ISTA</b>:
<div class = 'alert alert-info'> 
    
* $f(s) = \frac{1}{2}J_{\text{LS}}(s, y) \quad \Rightarrow  \quad \nabla f(s) =  H^T (H s - y)$,
* $g(s) = \lambda \|W s\|_1  \quad \Rightarrow  \quad \operatorname{prox}_{\gamma g}(z) = W^\top \operatorname{soft\_threshold}(W z, \gamma \lambda)$. 
</div> 
</div>

For **2 points**, complete the function `solve_wavelet` using the **Iterative Soft-Thresholding Algorithm (ISTA)**  to solve:
    $$s^\ast_\lambda = \arg \min_s J_{\text{LS}}(s,y) + 2 \lambda J_\text{reg}(s) = \arg \min_s \frac{1}{2} \|h * s - y\|_2^2 + \lambda  \|W s\|_1\,,$$
where $\lambda \geq 0$ is an hyperparameter.

The function `solve_wavelet` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 * `lam` (scalar): the hyperparameter $\lambda \geq 0$, 
 * `nb_iterations` (int): fixed to 50 (do not modify), 
 
and returns `solution`, the formal solution $s^\ast_\lambda$.

<div class="alert alert-danger">
    <b>Note:</b> The number of iterations is arbirarily set to $50$, even if the algorithm has not fully converged (do not modify it).
</div>

<div class="alert alert-info">
<b>Hint:</b> you will need to implement the soft thresholding function first:
$$\operatorname{soft\_threshold}(x, \tau) = \left\{
    \begin{array}{ll}
        x - \tau, & \text{ if } x > \tau \\
        x + \tau, & \text{ if } x < -\tau \\
        0, & \text{ otherwise }
    \end{array}
\right.$$
</div>

The function `soft_threshold` takes as inputs
 * `x`: a numpy array, 
 * `tau` (float): a threshold $\tau$, 
 
and returns the numpy array `x_thresholded`, where every elements of `x` have been soft-thresholded with threshold `tau`.

In [ ]:
def soft_threshold(x, tau):
    x_thresholded = None
    # YOUR CODE HERE
    return x_thresholded

In [ ]:
# Perform sanity check for soft_threshold
x_sanity_check = np.linspace(-2, 2, 11)
y_sanity_check = np.array([-1.,-0.6,-0.2,0.,0.,0.,0.,0.,0.2,0.6,1.])
if np.allclose(soft_threshold(x_sanity_check, 1.0), y_sanity_check):
    print('Congratulations! Your function "soft_threshold" seems to be working.') 
else:
    print('WARNING!\nYour function "soft_threshold" is not correct!')

In [ ]:
def solve_wavelet(y, h, lam, nb_iterations=50):
    s = y # initialization
    L = 1.0 # Lipschitz constant of data term
    gamma = 1/L
    for _ in range(nb_iterations):
        # YOUR CODE HERE
    solution = s
    return solution

In [ ]:
# Perform sanity check for solve_wavelet
np.random.seed(5678) # for reproductibility, do not modify
y_sanity_check = np.random.rand(16, 16)
h_sanity_check = np.array([[0.05, 0.1, 0.05], [0.1, 0.4, 0.1], [0.05, 0.1, 0.05]])
lam_sanity_check = 1e-1
solution_sanity_check = np.array([[0.46400174, 0.3931575, 0.43877703, 0.4990272],
       [0.54464673, 0.4738025, 0.4990272, 0.43877703],
       [0.87161648, 0.11777555, 0.49469602, 0.49469602],
       [0.87161648, 0.11777555, 0.49469602, 0.49469602]])
if np.allclose(solution_sanity_check, solve_wavelet(y_sanity_check, h_sanity_check, lam_sanity_check)[:4, :4]):
    print('Congratulations! Your function "solve_wavelet" seems to be working.') 
else:
    print('WARNING!\nYour function "solve_wavelet" is not correct!')

Observe the effet of the regularization hyperparameter $\lambda$ for deblurring Einstein image $y$.

<div class="alert alert-danger">
<b>Beware:</b> <code>solve_wavelet</code> takes about 5 seconds for 50 iterations.
</div>

In [ ]:
# Declare slider
reg_slider = widgets.IntSlider(value=-3, min=-3, max=8, step=1, description='lam = 10^x')
activation_button = widgets.Button(description='Regularize (wait 5s)')

# Declare callback
def callback(img):
    reg = reg_slider.value
    output = solve_wavelet(img, kernel_h_2D, 10**reg)
    return output

plt.close('all')
view = viewer(blurred_and_weakly_noisy_einstein, new_widgets=[reg_slider, activation_button], callbacks=[callback], widgets=True)

For **1 point**, answer the following MCQ:
* Among the following values of $\lambda$, which one maximizes the SNR between $s$ and $s^\ast_\lambda$ according to the ISTA algorithm (number of iterations fixed to 50) ? 
1. $\lambda = 10^{-1}$
2. $\lambda = 1$
3. $\lambda = 10^{1}$
4. $\lambda = 10^{2}$
5. $\lambda = 10^{3}$
6. $\lambda = 10^{4}$
7. $\lambda = 10^{5}$

Use the next cell to experiment or confirm your hypothesis and then modify the variable `answer` in the cell right after it to reflect your choices.  

In [ ]:
# YOUR CODE HERE

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

#### Behavior at the limits

**For 1 point** answer the following MCQ:
* What is $s_\lambda^\ast$ when $\lambda$ tends to infinity ? 

**Note:** $\mathbf{0}$ and $\mathbf{1}$ denote the all-zeros vector and the all-ones vector of the same size as $y$, respectively.

1. $y$
2. $\mathbf{0}$
3. $\mathbf{1}$
4. $Hy$
5. $H \mathbf{1}$
6. $\operatorname{mean}(y) \mathbf{1}$
7. $\operatorname{median}(y) \mathbf{1}$

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

### 2.C. TV regularization 
[Back to table of contents](#ToC_1_NN)

Another very popular type of regularization is the Total variation regularization $J_{\text{TV}}(s) = \lambda \| \nabla s\|_1$, which favors piecewise-constant solutions. Complete the function `solve_tv` using the **Proximal Gradient Descent** algorithm  to solve:
    $$s^\ast_\lambda = \arg \min_s \frac{1}{2} \|h * s - y\|_2^2 + \lambda  \| \nabla s\|_1\,,$$
where $\lambda \geq 0$ is an hyperparameter.

The function `solve_tv` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 * `lam` (scalar): the hyperparameter $\lambda \geq 0$, 
 * `nb_iterations` (int): fixed to 100 (do not modify), 
 
and returns `solution`, the formal solution $s^\ast_\lambda$.

<div class="alert alert-info">
<b>Hint:</b> use the function <a href="https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.denoise_tv_chambolle"><code>skimage.restoration.denoise_tv_chambolle</code></a> with the correct <code>weight</code> parameter for computing the proximal step as there exists no closed-form expression.
</div>
    
<div class="alert alert-danger">
<b>Beware:</b> since <a href="https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.denoise_tv_chambolle"><code>skimage.restoration.denoise_tv_chambolle</code></a> is also an iterative function, the algorithm may take some time.
</div>

In [ ]:
def solve_tv(y, h, lam, nb_iterations=100):
    s = y # initialization
    L = 1.0 # Lipschitz constant of data term
    gamma = 1/L
    for _ in range(nb_iterations):
        # YOUR CODE HERE
    solution = s
    return solution

In [ ]:
# Perform sanity check for solve_tv
np.random.seed(5678) # for reproductibility, do not modify
y_sanity_check = np.random.rand(16, 16)
h_sanity_check = np.array([[0.05, 0.1, 0.05], [0.1, 0.4, 0.1], [0.05, 0.1, 0.05]])
lam_sanity_check = 5e-2
solution_sanity_check = np.array([[0.37104013, 0.21921995, 0.39198927, 0.49997054],
       [0.5447789 , 0.48547633, 0.51157949, 0.50653231],
       [0.57900974, 0.46908567, 0.51937762, 0.52004448],
       [0.66397081, 0.46555742, 0.52212   , 0.57628085]])
if np.allclose(solution_sanity_check, solve_tv(y_sanity_check, h_sanity_check, lam_sanity_check)[:4, :4]):
    print('Congratulations! Your function "solve_tv" seems to be working.') 
else:
    print('WARNING!\nYour function "solve_tv" is not correct!')

Observe the effet of the regularization with hyperparameter $\lambda = 10^{-1}$ for deblurring Einstein image $y$.

<div class="alert alert-danger">
    <b>Beware:</b> <code>solve_tv</code> takes about 20 seconds for 100 iterations.
</div>

In [ ]:
lam = 1e-1
recovered_einstein = solve_tv(blurred_and_weakly_noisy_einstein, kernel_h_2D, lam, nb_iterations=100)

In [ ]:
plt.close('all')
view = viewer([recovered_einstein, blurred_and_weakly_noisy_einstein], widgets=True, hist=False, axis=True, cmap='gray')

#### Behavior at the limits

Answer the following MCQ:
* What is $s_\lambda^\ast$ when $\lambda$ tends to infinity ? 

**Note:** $\mathbf{0}$ and $\mathbf{1}$ denote the all-zeros vector and the all-ones vector of the same size as $y$, respectively.

1. $y$
2. $\mathbf{0}$
3. $\mathbf{1}$
4. $Hy$
5. $H \mathbf{1}$
6. $\operatorname{mean}(y) \mathbf{1}$
7. $\operatorname{median}(y) \mathbf{1}$

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

Solving inverse problems by adding a term of regularization is powerful but requires implementing a specific optimization algorithm for each form of the regularizer and the forward operator. In the second part of this lab, we are going to see that the library Pyxu, developed at EPFL, may be helpful for this. 